# Machine Learning with ibmdbpy and Db2 Warehouse:  Naïve Bayes Classification

This notebook explains how to perform Naive Bayes classification with ibmdbpy Python package and data stored in Db2 Warehouse.
___

### Accelerate Python analytics with in-database processing by using ibmdbpy and IBM Db2 Warehouse
 
The ibmdbpy project provides a Python interface for data manipulation and access to in-database algorithms in IBM Db2 Warehouse. It accelerates Python analytics by seamlessly pushing operations written in Python into the underlying database for execution, thereby benefitting from in-database performance-enhancing features, such as columnar storage and parallel processing. For more details about ibmdbpy, please refer to the [documentation](https://pythonhosted.org/ibmdbpy/index.html) and to the dedicated [Git repository](https://github.com/ibmdbanalytics/ibmdbpy/tree/master/ibmdbpy). This notebook provides you with an overview of ibmdbpy functionalities. 

 
__Prerequisites__
* Db2 account: see [IBM Cloud](https://cloud.ibm.com/login) or [Db2 Warehouse](https://www.ibm.com/support/knowledgecenter/en/SSCJDQ/com.ibm.swg.im.dashdb.kc.doc/welcome.html)
* Db2 driver: learn more on [IBM Knowledge Center](https://www.ibm.com/support/knowledgecenter/en/SSFMBX/com.ibm.swg.im.dashdb.doc/connecting/connect_applications_by_type.html) and see [IBM Support](https://www.ibm.com/support/pages/db2-jdbc-driver-versions-and-downloads)
* Having installed the [ibmdbpy](https://pypi.org/project/ibmdbpy/) python library with pip: 
> pip install ibmdbpy 
* Optional dependency for JDBC is the [jaydebeapi](https://pypi.org/project/JayDeBeApi/) library. Run the following command to install ibmdbpy, as well as the dependencies for the JDBC feature:
> pip install ibmdbpy[jdbc]

__About naive Bayes classification__

The Naive Bayes algorithm is a type of probabilistic ML classification algorithm. It is based on the Bayes theorem and on the assumption that predictors are independent. In practice, this is not really the case but naive Bayes classifiers perform quite well on tasks such as sentiment analysis, spam filtering and product recommendation.
More on the [IBM Knowledge Center](https://www.ibm.com/support/knowledgecenter/SSCJDQ/com.ibm.swg.im.dashdb.analytics.doc/doc/r_naive_bayes.html).

__Contents__

This notebook shows you how to perform naive Bayes classification with ibmdbpy on the iris data sample.

__1. Get started__
* Imports
* Establish a JDBC connection to Db2
* Load sample data

__2. Perform Naive Bayes Classification with ibmdbpy__
* Define an indexer
* Build the model
* Analyze the results

## 1. Get started

__Imports__

As usual we import some useful modules from ibmdbpy.

In [1]:
from ibmdbpy import IdaDataBase, IdaDataFrame
from ibmdbpy import learn
import pandas as pd

__Establish JDBC connection to Db2 Warehouse__

Please enter your credentials in order to connect to the database.

In [2]:
#Enter the values for you database connection
dsn_database = "___" # e.g. "BLUDB"
dsn_hostname = "___" # e.g.: "abc.url.example"
dsn_port = "___"    # e.g. "50000"
dsn_uid = "___"     # e.g. "db2_1234"
dsn_pwd = "___"     # e.g. "zorglub"

In [3]:
#Establish connection
connection_string='jdbc:db2://'+dsn_hostname+':'+dsn_port+'/'+dsn_database+':user='+dsn_uid+';password='+dsn_pwd+";" 
idadb=IdaDataBase(dsn=connection_string, verbose = False)
# set verbose to True if you want to see the detail of ibmdbpy operations

By default the environment variable `AUTOCOMMIT` is then set to True, which means that every SQL statement which is submitted through the connection is executed within its own transaction and then committed implicitly. When you close the connection to Db2, if the environment variable `AUTOCOMMIT` is set to False, then all changes after the last explicit commit are discarded. 

__Load sample data__

In this notebook, we simply use the well-known iris dataset, which is available out of the box in ibmdbpy. The dataset is first loaded as a Pandas DataFrame and then written into Db2 as a new table called IRIS. We define an IdaDataFrame with name idadf as a reference to this physical table. Finally, we add an ID column to index the rows.

In [4]:
# Fetch sample data
from ibmdbpy.sampledata import iris

# Convert the pandas DataFrame into an IdaDataFrame
idadf = idadb.as_idadataframe(iris, "IRIS", clear_existing=True)

idadf.head()

Uploading 150 rows (maxnrow was set to 20000)


,sepal_length,sepal_width,petal_length,petal_width,species
0,4.3,3.0,1.1,0.1,setosa
1,4.4,3.0,1.3,0.2,setosa
2,4.4,2.9,1.4,0.2,setosa
3,4.4,3.2,1.3,0.2,setosa
4,4.5,2.3,1.3,0.3,setosa


## 2. Perform Naive Bayes Classification with ibmdbpy

__Define an indexer__

You need an indexer to be able to fit the model on the dataset! We have defined the indexer with `add_column_id` in the previous cell. Alternatively, if an eligible column already exists, you can set it as indexer as follows:

> idadf = idadb.as_idadataframe(df, indexer = "col_name")

or, if the table already exists, define the IdaDataFrame by directly pointing at this table:

> idadf = IdaDataFrame(tablename, indexer = "col_name")

In [5]:
# Add an indexer column
# destructive = True ensures that the underlying table is modified too
# Otherwise, an ID column is only added to the view.
# Of course, physically modifying the table takes more time.
idadb.add_column_id(idadf, destructive = True)

In [6]:
print(idadf.indexer)
# sanity check

ID


In [7]:
idadf.head()

,ID,sepal_length,sepal_width,petal_length,petal_width,species
0,0,5.1,3.5,1.4,0.2,setosa
1,1,4.9,3.0,1.4,0.2,setosa
2,2,4.7,3.2,1.3,0.2,setosa
3,3,4.6,3.1,1.5,0.2,setosa
4,4,5.0,3.6,1.4,0.2,setosa


__Build the model__

In [8]:
# Define the model
NaiveBayesModel = learn.naive_bayes.NaiveBayes(modelname="NAIVE_IRIS", disc = 'ef', bins = 10)

In [9]:
# Inspect the model
NaiveBayesModel.get_params()

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_column_id', '_idadb', '_idadf', '_retrieve_NaiveBayes_Model', 'bins', 'coldefrole', 'coldeftype', 'colpropertiestable', 'describe', 'disc', 'fit', 'fit_predict', 'get_params', 'incolumn', 'labels_', 'mestimation', 'modelname', 'outtable', 'outtableProb', 'predict', 'set_params', 'target']


{'modelname': 'NAIVE_IRIS',
 'disc': 'ef',
 'bins': 10,
 'target': None,
 'incolumn': None,
 'coldeftype': None,
 'coldefrole': None,
 'colpropertiestable': None,
 'outtable': None,
 'outtableProb': None,
 'mestimation': None}

All of the above parameters are optional. 

* modelname: If not specified, will be automatically generated. If a model with the same name already exists, it will be replaced.

* disc: discretization type. Options are: Equal-frequency (ef), Minimal entropy (em), Equal-width (ew), Equal-width discretization with nice bucket limits (ewn).

* bins: int, number of buckets for discretization of numeric columns

* target: str, column of the input table that represents the class

* incolumn: list, columns of the input table that have specific properties. If None, then all columns have default properties. Properties can be about the type (continuous or nominal) or the role(id, target, to ignore, input).

More details on the [ibmdbpy documentation](https://pythonhosted.org/ibmdbpy/naive_bayes.html) and the [IBM Knowledge Center](https://www.ibm.com/support/knowledgecenter/SSCJDQ/com.ibm.swg.im.dashdb.analytics.doc/doc/r_naive_bayes.html)

In [10]:
# Fit the model to your dataset
NaiveBayesModel.fit(idadf, column_id="ID", target="species")

In [11]:
pred = NaiveBayesModel.predict(idadf, column_id = 'ID')

In [12]:
pred.head()

,ID,CLASS
0,0,setosa
1,1,setosa
2,2,setosa
3,3,setosa
4,4,setosa


__Analyze the results__

In [13]:
NaiveBayesModel.describe()

,species,setosa,versicolor,virginica
0,A-priori probabilities,0.33333,0.33333,0.33333


In [14]:
# confusion matrix
iris_df = idadf.as_dataframe().set_index('ID')
iris_df.rename(columns = {'species':'true_label'}, inplace = True)
pred_df = pred.as_dataframe().set_index('ID')
pred_df.rename(columns = {'CLASS':'pred_label'}, inplace = True)

pd.crosstab(pred_df['pred_label'], iris_df['true_label'])

true_label,setosa,versicolor,virginica
pred_label,,,
setosa,50,0,0
versicolor,0,48,5
virginica,0,2,45


In [15]:
# If you want more details:
NaiveBayesModel.describe(detail = True)

MODEL
       ATTRIBUTE VAL       CLASS  CLASSVALCOUNT  ATTRCLASSCOUNT  CLASSCOUNT  \
0   petal_length   1      setosa             50              50          50   
1   petal_length   2      setosa              0              50          50   
2   petal_length   3      setosa              0              50          50   
3   petal_length   3  versicolor              2              50          50   
4   petal_length   2  versicolor             48              50          50   
5   petal_length   1  versicolor              0              50          50   
6   petal_length   1   virginica              0              50          50   
7   petal_length   2   virginica              6              50          50   
8   petal_length   3   virginica             44              50          50   
9    petal_width   1      setosa             41              50          50   
10   petal_width   2      setosa              9              50          50   
11   petal_width   4      setosa              

,species,setosa,versicolor,virginica
0,A-priori probabilities,0.33333,0.33333,0.33333


1. `MODEL` table

The first table `MODEL` can be read as follows:

"In bucket VAL of column ATTRIBUTE, there are CLASSVALCOUNT items from CLASS out of CLASSCOUNT items from this class and ATTRCLASSCOUNT items of this class having this attribute."

Examples: 

* "In bucket 1 of column petal_length, there are 50 items from class setosa, out of 50 items from this class and 50 items from class setosa having this attribute."

* "In bucket 2 of column petal_length, there isn't any item from class setosa, out of 50 items from this class and 50 items having this attribute."

* "In bucket 3 of column petal_length, there are 44 items from class virginica, out of 50 items from this class and 50 items having this attribute." 


2. `TOTALCOUNT` table

Reminder: the iris dataset has 50 items from each of the 3 classes, so 150 in total. For each of the 36 (ATTRIBUTE, VAL, CLASS) triplet, the total number of items that fits in the buckets is given (not very insightful for a perfectly clean dataset like ours, but can be useful).

3. `DISCRANGE` table

The `DISCRANGE` table gives the borders of the discretization buckets for each feature. Note that the buckets are not listed in a specific order. For the `petal_length` attribute, we have 2 `BREAK` values, hence 3 buckets. Same for the other features. By default, numerical values are dicretized into 10 buckets or intervals for each class, id est each feature gets 9 breaking values.

__Congratulations!__ You are ready to perform naive Bayes classification with ibmdbpy!


## Where to go from here ?

More examples of ibmdbpy capabilities and ML algorithms are available on this GitHub repository.

* Getting started with ibmdbpy :
        
    [Basics](../GettingStarted/ibmdbpy_GettingStarted_1-basics.ipynb)

    [GeoBasics](../GettingStarted/ibmdbpy_GettingStarted_2-geo_basics.ipynb)

    [Extensive Guide](../GettingStarted/ibmdbpy_GettingStarted_3-geo_guide.ipynb)
    

* More practice : analyze the Museums dataset, understand how to create IdaDataFrames and IdaGeoDataFrames:
        
    [Preprocessing](../MuseumsUseCase/ibmdbpy_Museums_DataAnalysis_1-preprocessing.ipynb)

    [Geospatial recommendation](../MuseumsUseCase/ibmdbpy_Museums_DataAnalysis_2-geospatial.ipynb)


* Machine learning with ibmdbpy: 
        
    [Association Rules Mining](../MachineLearning/ibmdbpy_AssociationRulesMining.ipynb)

    [KMeans](../GettingStarted/ibmdbpy_GettingStarted_1-basics.ipynb) 

_____
__Author__

Eva Feillet - ML intern, IBM Cloud and Cognitive Software @ IBM Research & Development Germany, Böblingen lab.